## LSTM 머신 모델 테스트

In [2]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random, sys, csv

# CSV 파일 읽기
csv_file = "new_indie.csv"  # CSV 파일 경로
text = ""

with open(csv_file, "r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        text += row[3] + " "  # 3 번째 열의 데이터를 이어붙임

print('코퍼스의 길이: ', len(text))

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))  # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars))  # ID → 문자

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1)

    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)

    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)

        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


코퍼스의 길이:  447162
사용되고 있는 문자의 수: 1320
학습할 구문의 수: 149048
텍스트를 ID 벡터로 변환합니다...
모델을 구축합니다...

--------------------------------------------------
반복 = 1


c:\wanted_l\ML\env311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1165/1165 ━━━━━━━━━━━━━━━━━━━━ 54s 45ms/step - loss: 4.1561

--- 다양성 =  0.2
--- 시드 = "일 어쩔 땐
다른 생각도 못 해
나만"
일 어쩔 땐
다른 생각도 못 해
나만 가득 없어서 말해

그대가 내 밤이 가득 걸
내가 가득 말해

그대 내 마음 속에 있어

그대로 가득 말해
나를 가득 말해
 그 가득 있어

우리가 가득 수 있을까

그대 내 마음 말해 
내가 가득 말해
어디 가 가득 있어

그대가 나의 내 마음을 가득해

그대로 나를 가득 있어

우리가 나를 가득 걸 그대
나를 가득해 내 마음을 가득해

그대 내가 가득 말 말 걸 

그대가 가득 너를 가득해

그대 너를 가득 있을까

그대로 가득 말 수 있을까
그대로 가득 수 있을 때 수 있어

그대 내가 가득 그 때 있어

우리가 너를 가득 때 가득 말 좋아
너를 가득 밤 그대가 있어

그대로 나 나 있어

그대 내 가득 있어

그대의 너의 밤 너를 가득 있고

우리가 좋아하는 너를 있어

나를 가득 말해
우리가 가득

--- 다양성 =  0.5
--- 시드 = "일 어쩔 땐
다른 생각도 못 해
나만"
일 어쩔 땐
다른 생각도 못 해
나만 내 다시 있어
내가 모르게 나 좋아
 
텨 그 걸 말 말 않게
나만 그대가 내가 그 살아
그대로 그대로 내게 
매 내 더 가득 때 모습이 하나
내가 지나가 가득 말
그대 기억하게 걸 걸

아음은 가득 내리 사랑해
너를 위해 너를 수 있을까
우리가 가득 말 하루게 있어

그대로 나는 밤 것 맙게 가 가려가 나

그대로 나는 그리가 가득 하나가 있어
우리가 우리가 가득가 싶어
전에 우리가 나를 내리게 마
내게 사랑이 가는 봐

그렇게 가득 때 있어

아무 때러 가득 말 수 있는데
우리 없는 내 건 때
어디한 떠나가 돼
너를 마음이 다 좋아
돌아서 안 속에 
지나가 가득 싶어

우리를 가득한 수 있고 싶어
내 밤이 가득 하루가 싶어

우리가 가득 위해 가는 걸
내 마음이 나를 있어
또 마음에 기억해
더 그대가 

-

KeyboardInterrupt: 